In [230]:
import cv2
import numpy as np

def get_obstacles(img):

    field               = {"obstacles":[],"start":None,"end":None}
    gray                = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret,thresh          = cv2.threshold(gray,50,255,0)
    contours,hierarchy  = cv2.findContours(thresh, 1, 2)

    for cnt in contours:
        x1,y1 = cnt[0][0]
        approx = cv2.approxPolyDP(cnt, 0.01*cv2.arcLength(cnt, True), True)
        if len(approx) == 4:
            x, y, w, h = cv2.boundingRect(cnt)
            ratio = float(w)/h
            if ratio >= 0.8 and ratio <= 1.2:
                img = cv2.drawContours(img, [cnt], -1, (0,255,255), 3)
                if field["start"]==None:field["start"]=(int(x+w/2),int(y+h/2),y)
                else:
                    if y<field["start"][2]:
                        field["end"]=field["start"]
                        field["start"]=(int(x+w/2),int(y+h/2),y)
                    else:
                        field["end"]=(int(x+w/2),int(y+h/2),y)
            else:
                cv2.putText(img, 'Obstacle', (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
                img = cv2.drawContours(img, [cnt], -1, (0,255,0), 3)
                field["obstacles"].append((x,y,w,h))
    cv2.imwrite("temp_obstacle.png",img)
    return field,img



In [231]:
def find_way(obstacle,curr_x,curr_y,expected_x,expected_y,img):
    # Return me to same x coordinate but after crossing obstacle in y coordinate.
    dist_left = curr_x - obstacle[0]
    dist_right = obstacle[0]+obstacle[2]-curr_x
    tot_dist_left = 2*dist_left + expected_x-curr_x
    tot_dist_right = 2*dist_right + curr_x - expected_x
    #print("dist_left,rigth",dist_left,dist_right)
    if tot_dist_left<tot_dist_right:
        cv2.line(img,(curr_x,curr_y),(curr_x-dist_left-5,curr_y),(255,0,0),2)
        cv2.line(img,(curr_x-dist_left-5,curr_y),(curr_x-dist_left-5,curr_y+obstacle[3]+10),(255,0,0),2)
        cv2.line(img,(curr_x-dist_left-5,curr_y+obstacle[3]+10),(expected_x,expected_y),(255,0,0),2)
    else:
        cv2.line(img,(curr_x,curr_y),(curr_x+dist_right+5,curr_y),(255,0,0),2)
        cv2.line(img,(curr_x+dist_right+5,curr_y),(curr_x+dist_right+5,curr_y+obstacle[3]+10),(255,0,0),2)
        cv2.line(img,(curr_x+dist_right+5,curr_y+obstacle[3]+10),(expected_x,expected_y),(255,0,0),2)
    return img


In [232]:
def get_path(img,field):
 

    heuristic_slope = (field["end"][0]-field["start"][0])/(field["end"][1]-field["start"][1])
    intercept = field["start"][0]
    #print(heuristic_slope)

    curr_x = int(field["start"][0])
    curr_y = int(field["start"][1])

    flag = True
    while(flag):
        next_point = (int(10*heuristic_slope + curr_x),curr_y+10)

        for (x,y,w,h) in field["obstacles"]:
            if next_point[0]>x and next_point[0]<x+w and next_point[1]>y and next_point[1]<y+h:
                expected_y = curr_y+h+8
                expected_x = int((h+8)*heuristic_slope + curr_x)
                img = find_way((x,y,w,h),curr_x,curr_y,expected_x,expected_y,img)
                curr_x = expected_x
                curr_y = expected_y
                heuristic_slope = (field["end"][0]-curr_x)/(field["end"][1]-curr_y)
                next_point = (int(10*heuristic_slope + curr_x),curr_y+10)
                #display(curr_x,curr_y)
                #print(heuristic_slope)
        #if (curr_x-field["end"][0]<=5 or field["end"][0]-curr_x<=5) and (curr_y-field["end"][1]<=5 or field["end"][1]-curr_y<=5):flag=False
        if curr_y>field["end"][1]:flag=False
        if flag:
            #print("(",curr_x,",",curr_y,") - (",next_point[0],",",next_point[1],")")
            cv2.line(img,(curr_x,curr_y),(next_point[0],next_point[1]),(255,0,0),2)
        curr_x = int(next_point[0])
        curr_y = int(next_point[1])

    cv2.imwrite("temp_path.png",img)
    return img

In [238]:

img                     =       cv2.imread('./temp6.png')
field,obstacle_img      =       get_obstacles(img)
#print(field)
path_img                =       get_path(obstacle_img,field)
